In [1]:
import os
os.cpu_count()

4

In [2]:
from multiprocessing import Pool
def sample_func(initial_num):
    for i in range(100):
        initial_num += 1
    return initial_num


initial_num_list = list(range(10))
with Pool(processes=4) as p:
    result_list = p.map(func=sample_func, iterable=initial_num_list)
print(result_list)

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109]


In [3]:
import time
from multiprocessing import Pool

In [4]:
def nijou(x, y):
    return [x**2, y**2]

def nijou_wrapper(args):
    return nijou(*args)

In [5]:
%%time
if __name__ == "__main__":
    p = Pool(4)
    values = [(x, y) for x in range(4) for y in range(4)]
    result = p.map(nijou_wrapper, values)
result

CPU times: user 4.98 ms, sys: 11.8 ms, total: 16.8 ms
Wall time: 16.5 ms


[[0, 0],
 [0, 1],
 [0, 4],
 [0, 9],
 [1, 0],
 [1, 1],
 [1, 4],
 [1, 9],
 [4, 0],
 [4, 1],
 [4, 4],
 [4, 9],
 [9, 0],
 [9, 1],
 [9, 4],
 [9, 9]]

Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-7:
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, 

In [6]:
%%time
if __name__ == "__main__":
    values = [(x, y) for x in range(4) for y in range(4)]
    result = [nijou(x,y) for x,y in values]
result

CPU times: user 45 µs, sys: 5 µs, total: 50 µs
Wall time: 55.1 µs


[[0, 0],
 [0, 1],
 [0, 4],
 [0, 9],
 [1, 0],
 [1, 1],
 [1, 4],
 [1, 9],
 [4, 0],
 [4, 1],
 [4, 4],
 [4, 9],
 [9, 0],
 [9, 1],
 [9, 4],
 [9, 9]]

In [2]:
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from InfMaxProblem import Monte_Carlo as mc
import collections
from multiprocessing import Pool

In [3]:
# データの読み込み

# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data.csv")
network.head()

# numpy型に変換
network_np = network.values

# 空の有向グラフを作成
G = nx.DiGraph()

# 重み付きの枝を加える
G.add_weighted_edges_from(network_np)

In [4]:
def experiment_IC(G, seed, T=10000):
    inf_sum = 0
    inf_size_list = []
    for i in range(T):
        simulation = mc.IC_simulation(G, seed)
        
        # 影響数
        inf_sum += simulation
        inf_size_list.append(inf_sum / (i+1))
    return inf_sum/T

In [5]:
def IC_simulation_wrapper(args):
    return mc.IC_simulation(*args)

In [36]:
def experiment_IC_mult(G, seed, T=10000):
    with Pool(processes=4) as p:
        values = [(G, seed) for t in range(T)]
        result = p.map(IC_simulation_wrapper, values)
    return sum(result)/T

In [28]:
# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data.csv")
network.head()

# numpy型に変換
network_np = network.values

# 空の有向グラフを作成
G = nx.DiGraph()

# 重み付きの枝を加える
G.add_weighted_edges_from(network_np)

seed = pd.DataFrame({"data":[i for i in range(10)]})
seed = pd.DataFrame({"WC":[i for i in range(10)]}).values.reshape(-1,)

In [29]:
%time experiment_IC_mult(G, seed, T=10)

CPU times: user 30.5 ms, sys: 69 ms, total: 99.5 ms
Wall time: 201 ms


[22, 22, 22, 22, 16, 16, 16, 16, 17, 17]

# Epinions

In [30]:
# resultフォルダからseedを取り出す
seed_df = pd.read_csv("result/Epinions/seed.csv")

## WC

In [31]:
# 枝確率を計算済みのネットワークを読み込む
network = pd.read_csv("data/Epinions/WC.csv")
network.head()

,# FromNodeId,ToNodeId,WC
0,0,4,0.008000
1,0,5,0.005682
2,0,7,0.033333
3,0,8,0.009615
4,0,9,0.066667


In [32]:
# numpy型に変換
network_np = network.values

In [33]:
G = nx.DiGraph()
G.add_weighted_edges_from(network_np)

In [34]:
seed = seed_df["WC"].values
seed

array([7.6300e+02, 6.3400e+02, 6.4500e+02, 5.2320e+03, 7.1399e+04,
       3.9240e+03, 1.8350e+03, 5.2270e+03, 6.3700e+02, 2.9690e+03,
       1.0590e+03, 1.4500e+02, 4.9200e+03, 2.0660e+03, 1.2250e+03,
       7.0470e+03, 1.7200e+03, 2.7040e+03, 1.5960e+03, 9.4120e+03,
       1.5390e+03, 5.2240e+03, 1.6690e+03, 6.2900e+02, 1.5330e+03,
       4.4000e+01, 5.1440e+03, 8.2400e+02, 6.0030e+03, 7.7000e+02,
       1.1720e+03, 6.7740e+03, 2.1180e+03, 1.4090e+03, 3.8500e+03,
       7.1388e+04, 1.5010e+03, 3.5270e+03, 1.6248e+04, 6.6100e+02,
       7.3920e+03, 4.9310e+03, 6.5000e+02, 5.3820e+03, 5.4600e+02,
       6.4890e+03, 3.4150e+03, 1.7270e+03, 7.3940e+03, 7.6870e+03])

In [38]:
%time experiment_IC_mult(G, seed, T=10000)

CPU times: user 1min 29s, sys: 4.77 s, total: 1min 34s
Wall time: 17min 52s


12073.4456

In [ ]:
%time experiment(G, seed, T=10000)

In [29]:
inf_size_list = []
for s in tqdm(range(0,51,2)):
    inf_size = experiment_IC_mult(G, seed[:s])
    inf_size_list.append(inf_size)

/home/fueki/anaconda3/envs/all/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Process ForkPoolWorker-60:


KeyboardInterrupt: 

Process ForkPoolWorker-57:
Process ForkPoolWorker-59:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/proce

  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/fueki/anaconda3/envs/all/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [ ]:
pd.DataFrame(inf_size_list, index=range(0,51,2)).plot()